# リスクリターン平面の作成

In [1]:
import toml
import pandas as pd
import datetime
import copy
import numpy as np
from scipy.stats import gmean

import yfinance as yf

from statsmodels.tsa.seasonal import seasonal_decompose

import seaborn as sns
import matplotlib.pyplot as plt

from pptx import Presentation
from pptx.util import Cm, Pt, Inches  # 単位変換機能
from pptx.dml.color import RGBColor   # 色指定用

from deal import *
from load_ts import *

## 時系列の取得

In [2]:
tickers = [ 'VOO'  # S&P500 連動 ETF
          , 'SPYD' # S&P500 高配当株式 ETF
          , 'VYM'  # バンガード米国高配当株式 ETF
          , 'SCHD' # シュワブ米国高配当株式 ETF
          , 'VT'   # バンガード・トータル・ワールドストックETF
          , 'GLD'  # SDPRゴールドシェア
         ]
names = [ 'ｅＭＡＸＩＳ　Ｓｌｉｍ全世界株式（オール・カントリー）'
        , 'ｅＭＡＸＩＳ　Ｓｌｉｍ米国株式（Ｓ＆Ｐ５００）'
        , 'ＭＡＸＩＳトピックス上場投信'
        , 'ＭＡＸＩＳトピックス上場投信'
        , 'ＮＥＸＴ　ＦＵＮＤＳ金価格連動型上場投信'
        , 'インデックスコレクション（国内債券）'
        , 'インデックスコレクション（外国債券）'
        , 'たわらノーロード全世界株式'
        , '上場インデックスファンドＴＯＰＩＸ'
        , '上場インデックスファンド海外先進国株式（ＭＳＣＩ－ＫＯＫＵＳＡＩ）'
        , '上場インデックスファンド米国株式（Ｓ＆Ｐ５００）'
        , '東京海上セレクション・物価連動国債'
        ]

In [3]:
#close_df, div_df = pick_yfinance_data(labels)
#div_rate_df = div_df/close_df

close_df, _ = pick_csv_data(['ｅＭＡＸＩＳ　Ｓｌｉｍ米国株式（Ｓ＆Ｐ５００）',])

## テストコード

### テストデータ

以下の eMAXIS Slim 米国株式のリスクリターンを正確に計算するよう計算式を組み立てる

![](sp500.png)

In [9]:
y0 = close_df.index[0].year

### 年率リターンの計算 ← 完了

日経の年率リターンの計算式\[1\]によれば，
$n$ヶ月前の月次の終値を$X_n$とした時，$n$ヶ月前の月次リターンを$p_n$とすると，
$$p_n=\frac{X_n-X_{n-1}}{X_{n-1}}$$
ここで，$N$ヶ月の累積リターン$\bar{p}$は
$$\bar{p}=\prod^N_{k=1}(1+p_k)-1$$
この時，年率リターンは
$$(1+\bar{p})^{12/N}-1$$

\[1\] [日本経済新聞 ヘルプセンター](https://www.nikkei.com/help/contents/markets/fund/#qf15)

In [8]:
target = close_df['ｅＭＡＸＩＳ　Ｓｌｉｍ米国株式（Ｓ＆Ｐ５００）']
target.loc[target.index[-1]+datetime.timedelta(days=3)] = target.iloc[-1]  # 2月末までしかデータがないので，1営業日のばす
for N in [6,12,36,60]:
    # diffは先頭がNaNになることを利用し，月始めにTrueとなる判定式を1つシフト
    # 結果，月次の終値の系列となる
    monthly = target[:-1][target.index.month.diff().dropna()!=0]
    returns = monthly.pct_change().dropna()[-N:]  # 月次リターン
    N = min(N,returns.size)  # 系列が不足している場合に系列超に上書き
    print(f'N={N}, return = {(returns+1).prod()**(12/N)-1:.4f}')  # Nヶ月の年率リターン

N=6, return = 0.1862
N=12, return = 0.1610
N=36, return = 0.2161
N=60, return = 0.2346


### リスクの計算 ← 1年以上はOK

日経の年率リスクの計算式\[2\]によれば，6カ月は日次データ、1年は週次データ、3年超は月次データで計算している
n週前の週次の終値を$X_n$とした時，n週前のリターン $p_n$ の計算は
$$p_n=\frac{X_n-X_{n-1}}{X_{n-1}}$$
ここで，$N$週分のリターンから年率のリスクを計算する式は
$$\frac{\sqrt{N\sum_k (p_k)^2-(\sum_kp_k)^2}}{N(N-1)}\times \sqrt{52}$$

\[2\] [日本経済新聞 ヘルプセンター](https://www.nikkei.com/help/contents/markets/fund/#qf17)

In [22]:
target = close_df['ｅＭＡＸＩＳ　Ｓｌｉｍ米国株式（Ｓ＆Ｐ５００）']
target.loc[target.index[-1]+datetime.timedelta(days=3)] = target.iloc[-1]  # 2月末までしかデータがないので，1営業日のばす
for period in [0.5, 1, 3, 5]:
    if period == 1:
        N = 52
        # weekly
        weekly = target.iloc[:-1][target.index.weekday.diff().dropna()<=0] 
        returns = weekly.pct_change().dropna()[-N:]
        # diffは先頭がNaNになることを利用，曜日番号の差分系列は週末を越えると負になることを利用，週始めにTrueとなる判定式を1つシフト
        # 年末年始などで4営業日がつぶれる場合は ==0 も含むので判定式は <= とする
        # 結果，週次の終値の系列となる
        weekly = target.iloc[:-1][target.index.weekday.diff().dropna()<=0] 
        returns = weekly.pct_change().dropna()[-N:]
        sqrt_N = np.sqrt(52)
    elif period > 1:
        N = period * 12  # 3年～は月次リターンの分散から計算
        # monthly
        monthly = target[:-1][target.index.month.diff().dropna()!=0]
        returns = monthly.pct_change().dropna()[-N:]  # 月次リターン
        sqrt_N = np.sqrt(12)
    elif period == 0.5:
        y,m,d = target.index[-2].timetuple()[:3]
        if m-6 > 0:
            m -= 6
        else:
            y -= 1
            m += 6
        tmp = target.loc[datetime.datetime(y,m,d):]
        returns = tmp.pct_change().dropna()
        display(returns.size)
        display(target.index[-2].timetuple()[:3],(y,m,d))
        sqrt_N = np.sqrt(252)
    #N = min(N,returns.size)  # 系列が不足している場合に系列超に上書き
    print(f'N={N}, risk={returns.std()*sqrt_N}')  # 6ヶ月の日次が合わない，1年の週次は合った，3年の月次は未

122

(2025, 2, 28)

(2024, 8, 28)

N=60, risk=0.17566970165030074
N=52, risk=0.16743513216461686
N=36, risk=0.16860986407504724
N=60, risk=0.17138583002061025


In [28]:
yearly_sp500.pct_change()

2020-02-27         NaN
2021-02-26    0.200770
2022-02-28    0.259275
2023-02-28    0.086462
dtype: float64

In [14]:
arithmetic_mean = yearly_sp500.pct_change().dropna().mean()
geometric_mean = ((yearly_sp500.iloc[-1]-yearly_voo.iloc[0])/yearly_voo.iloc[0])**(1/(len(yearly_voo)-1))-1
arithmetic_mean, geometric_mean

(np.float64(0.2084201943304944), np.float64(0.00385824758162312))

In [7]:
returns = yearly_voo.pct_change().dropna()
risk = returns.std()
risk

np.float64(0.16686599278995107)

In [8]:
daily_return = close_df['VOO'].pct_change().dropna()
daily_arithmetic_mean = daily_return.mean()
daily_risk = daily_return.std()

In [9]:
daily_arithmetic_mean, daily_risk

(np.float64(0.0005996742373635516), np.float64(0.011277977223254496))

In [10]:
daily_arithmetic_mean*252, arithmetic_mean

(np.float64(0.151117907815615), np.float64(0.15672153845471978))

In [11]:
daily_risk*np.sqrt(252), risk

(np.float64(0.17903233814749306), np.float64(0.16686599278995107))

In [12]:
year_voo = close_df['VOO'].loc[datetime.datetime(2024,2,1):datetime.datetime(2025,1,31)]
returns = year_voo.pct_change().dropna()
print('risk is ',returns.std()*np.sqrt(len(returns)))

risk is  0.1271528542858593


In [13]:
# 0 ... 月曜, 6 ... 日曜
# 金曜のサンプル close_df.index[1] == Timestamp('2015-10-23 00:00:00')
# 曜日番号系列 close_df.index.weekday
# 曜日番号の差分で、負の値をとるところが曜日またぎ close_df.index.weekday.diff().dropna()
# 先頭[0]は「金曜日-木曜日」なので正、2番目[1]は「月曜日-金曜日」なので負、これがindex[1]と一致するので、これで週の最終営業日の終値を抜き出せた
weekly_voo = close_df['VOO'].iloc[:-1][close_df.index.weekday.diff().dropna()<0]
# 1月末時点の1年分の週次分析
tmp = weekly_voo.loc[datetime.datetime(2024,2,1):datetime.datetime(2025,1,31)]
weekly_returns = tmp.pct_change().dropna()
print('length of series is ', len(weekly_returns))
print('arithmetic mean return is ', weekly_returns.mean() * len(weekly_returns))
print('weekly risk is ', weekly_returns.std()*np.sqrt(len(weekly_returns)))

length of series is  52
arithmetic mean return is  0.2186018129112031
weekly risk is  0.12692074233615888


In [14]:
sq_sum = (weekly_returns*weekly_returns).sum()  # 平方和
sum_sq = weekly_returns.sum()**2  # 和の平方
variance = (len(weekly_returns)*sq_sum - sum_sq)/len(weekly_returns)/(len(weekly_returns)-1)  # 不偏分散
std = np.sqrt(variance)  # 標準偏差
std*np.sqrt(len(weekly_returns))

np.float64(0.1269207423361589)

In [30]:
import csv
import os
import re

jpdate_splitter = re.compile('[年月日]')
index_column = '年月日'
data_column = '基準価額(円)'

def to_date(datestr):
    '''input is date string joined by date_splitter

    now only jp date format is implemented.
    '''
    if datestr.find('年') > 0:
        date_list = jpdate_splitter.split(datestr)
        if len(date_list) < 3:
            raise Exception(f'invalid date string: {datestr}')
        return datetime.datetime(*list(map(int, date_list[:3])))
    else:
        raise Exception(f'unknown date format: {datestr}')

def load_csv(mutual_fund_name):
    if os.path.isfile(mutual_fund_name+'.csv'):
        data = pd.read_csv(mutual_fund_name+'.csv', encoding='932')
        #with open(mutual_fund_name+'.csv') as fobj:
        #    cobj = csv.reader(fobj.read().rstrip().splitlines())
        #    data = [row for row in cobj]
    else:
        raise Exception(f'No such mutual fund data: {mutual_fund_name}')
    data[index_column] = data[index_column].apply(to_date)
    series = data.set_index(index_column)[data_column]
    series.name = mutual_fund_name
    return series

emaxis_slim_sp500 = load_csv('ｅＭＡＸＩＳ　Ｓｌｉｍ米国株式（Ｓ＆Ｐ５００）')

In [32]:
year_sp500 = emaxis_slim_sp500.loc[datetime.datetime(2024,2,1):datetime.datetime(2025,1,31)]
returns = year_sp500.pct_change().dropna()
print('risk is ',returns.std()*np.sqrt(len(returns)))

risk is  0.1743928881301315


In [37]:
# 0 ... 月曜, 6 ... 日曜
# 金曜のサンプル close_df.index[1] == Timestamp('2015-10-23 00:00:00')
# 曜日番号系列 close_df.index.weekday
# 曜日番号の差分で、負の値をとるところが曜日またぎ close_df.index.weekday.diff().dropna()
# 先頭[0]は「金曜日-木曜日」なので正、2番目[1]は「月曜日-金曜日」なので負、これがindex[1]と一致するので、これで週の最終営業日の終値を抜き出せた
weekly_sp500 = emaxis_slim_sp500.iloc[:-1][emaxis_slim_sp500.index.weekday.diff().dropna()<0]
# 1月末時点の1年分の週次分析
tmp = weekly_sp500.loc[datetime.datetime(2024,2,1):datetime.datetime(2025,1,31)]
weekly_returns = tmp.pct_change().dropna()
print('length of series is ', len(weekly_returns))
print('arithmetic mean return is ', weekly_returns.mean() * len(weekly_returns))
print('weekly risk is ', weekly_returns.std()*np.sqrt(len(weekly_returns)))
print('geometric mean return is ', ((year_sp500.iloc[-1]-year_sp500.iloc[0])/year_sp500.iloc[0])) #← 日経と合わせるなら、月次リターンで計算しないといけない

length of series is  51
arithmetic mean return is  0.2906505308484385
weekly risk is  0.16296403772051915
geometric mean return is  0.3323000863219022


参考 → [株価のリターンを計算する：幾何平均リターンとリスク｜yo4shi80](https://note.com/yo4shi80/n/n356ab23a06ee#38766d98-3bd2-4a24-bbb0-3eb0eda8cd56)

In [30]:
yearly_voo,gmean(yearly_voo)

(2015-12-31    159.721619
 2016-12-30    179.154541
 2017-12-29    218.162888
 2018-12-31    208.344086
 2019-12-31    273.691254
 2020-12-31    323.843445
 2021-12-31    417.092438
 2022-12-30    341.289917
 2023-12-29    431.130676
 2024-12-31    538.809998
 dtype: float64,
 np.float64(287.15408225345277))

In [31]:
prev = 1
out = [1]
for i in range(10):
    prev *= 1.01
    out.append(prev)
print(out)
print(gmean(out))

[1, 1.01, 1.0201, 1.030301, 1.04060401, 1.0510100501, 1.061520150601, 1.0721353521070098, 1.08285670562808, 1.0936852726843609, 1.1046221254112045]
1.0510100501


In [33]:
(out[-1]/out[0])**(1/(len(out)-1))

1.01